# ENCen40

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.ENCen40)

class ENCen40(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        return x



In [3]:
model = pya.models.ENCen40()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'encen40'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2023
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Dec, Eric, et al. \"Centenarian clocks: epigenetic clocks for validating claims of exceptional longevity.\" GeroScience (2023): 1-19."
model.metadata["doi"] = 'https://doi.org/10.1007/s11357-023-00731-7'
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download GitHub repository

In [5]:
github_url = "https://github.com/victorychain/Centenarian-Clock.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

## Load features

#### From CSV file

In [6]:
df = pd.read_csv('Centenarian-Clock/clocks/final_clocks.csv', index_col=0).T
df = df[df['ENCen40+'] != 0]
df = df.reset_index()

model.features = df['index'][1:].tolist()

## Load weights into base model

In [7]:
weights = torch.tensor(df['ENCen40+'][1:].tolist()).unsqueeze(0).float()
intercept = torch.tensor([df['ENCen40+'][0]]).float()

#### Linear model

In [8]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [9]:
model.reference_values = None

## Load preprocess and postprocess objects

In [10]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [11]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [12]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Dec, Eric, et al. "Centenarian clocks: epigenetic clocks for '
             'validating claims of exceptional longevity." GeroScience (2023): '
             '1-19.',
 'clock_name': 'encen40',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1007/s11357-023-00731-7',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2023}
reference_values: None
preprocess_name: None
preprocess_dependencies: None
postprocess_name: None
postprocess_dependencies: None
features: ['cg06437747', 'cg12140144', 'cg17804348', 'cg18025409', 'cg17489312', 'cg15952725', 'cg20822990', 'cg09195271', 'cg18107827', 'cg14333454', 'cg08169949', 'cg25410668', 'cg15554401', 'cg04299080', 'cg12065799', 'cg23093580', 'cg07388493', 'cg13806070', 'cg03664992', 'cg12671744', 'cg26881761', 'cg06784991',

## Basic test

In [13]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ 53.0407],
        [109.0838],
        [  9.5339],
        [ 91.3573],
        [ -9.1270],
        [100.4752],
        [ 43.2027],
        [ 84.4988],
        [-64.7365],
        [110.1437]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [14]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [15]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted folder: Centenarian-Clock
